In [3]:
# unit test
import unittest
# test data
import medmnist
from medmnist import INFO

from PIL import Image, ImageDraw

# "helper" needs to be part of sys path
import sys
sys.path.insert(0, "helper")
sys.path.insert(0, "/helper")
sys.path.insert(0, "./helper")
sys.path.insert(0, "../helper")
print(sys.path)

%load_ext autoreload
%autoreload 2

# own module
from loss import * 

['../helper', './helper', '/helper', 'helper', './helper', '/helper', 'helper', './helper', '/helper', 'C:\\Users\\Prinzessin\\projects\\ogygia_torch\\medical_ai\\examples', 'C:\\Users\\Prinzessin\\anaconda3\\envs\\feta\\python39.zip', 'C:\\Users\\Prinzessin\\anaconda3\\envs\\feta\\DLLs', 'C:\\Users\\Prinzessin\\anaconda3\\envs\\feta\\lib', 'C:\\Users\\Prinzessin\\anaconda3\\envs\\feta', '', 'C:\\Users\\Prinzessin\\anaconda3\\envs\\feta\\lib\\site-packages', 'C:\\Users\\Prinzessin\\anaconda3\\envs\\feta\\lib\\site-packages\\win32', 'C:\\Users\\Prinzessin\\anaconda3\\envs\\feta\\lib\\site-packages\\win32\\lib', 'C:\\Users\\Prinzessin\\anaconda3\\envs\\feta\\lib\\site-packages\\Pythonwin', 'C:\\Users\\Prinzessin\\anaconda3\\envs\\feta\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\Prinzessin\\.ipython']
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
class UnitTestLoss(unittest.TestCase):
    
    def test_kappa_loss(self):
        # =============================================================================
        # testing the Kappa loss function
        # =============================================================================
        n_classes = 3
        criterion = KappaLoss(n_classes=n_classes)

        model_outputs = [torch.tensor([[-0.1, -0.1, 5], [4, -0.1, -0.1], [-0.1, 6, -0.1], [-0.1, -0.1, 3]]), # good
                         torch.tensor([[0.1, 0.1, 0.9], [0.9, 0.1, 0.1], [0.1, 0.9, 0.1], [0.1, 0.1, 0.9]]), # middle
                         torch.tensor([[5, 0.1, 0.9], [0.9, 3, 0.1], [0.1, -0.3, 0.7], [0.7, 0.1, 0.1]])  # bad
                        ]

        ground_truth = torch.tensor([2, 0, 1, 2])

        print('='*50)
        print("Kappa Loss")
        for model_output in model_outputs:
            loss = criterion(model_output=model_output, ground_truth=ground_truth)
            print(round(loss.item(), 3))
            
    def test_cdw_loss_v1(self):
        # =============================================================================
        # testing the Class Distance Weighted loss function
        # which version is most efficient - check for 100 loops e.g.
        # =============================================================================
        n_classes = 3
        criterion = CDWLossChrisy(n_classes=n_classes)

        model_outputs = [  torch.tensor([[-0.1, -0.1, 5], [4, -0.1, -0.1], [-0.1, 6, -0.1], [-0.1, -0.1, 3]]), # good
                            torch.tensor([[0.1, 0.1, 0.9], [0.9, 0.1, 0.1], [0.1, 0.9, 0.1], [0.1, 0.1, 0.9]]), # middle
                            torch.tensor([[5, 0.1, 0.9], [0.9, 3, 0.1], [0.1, -0.3, 0.7], [0.7, 0.1, 0.1]])  # bad
        ]

        ground_truth = torch.tensor([2, 0, 1, 2])

        print('='*50)
        print("Class Weighted Distance 1")
        for model_output in model_outputs:
            loss = criterion(model_output=model_output, ground_truth=ground_truth)
            print(round(loss.item(), 3))
            
    def test_cdw_loss_v1_1(self):
        # =============================================================================
        # testing the loss function
        # =============================================================================
        n_classes = 3
        criterion = CDWLossRam(n_classes=n_classes)

        model_outputs = [  torch.tensor([[-0.1, -0.1, 5], [4, -0.1, -0.1], [-0.1, 6, -0.1], [-0.1, -0.1, 3]]), # good
                            torch.tensor([[0.1, 0.1, 0.9], [0.9, 0.1, 0.1], [0.1, 0.9, 0.1], [0.1, 0.1, 0.9]]), # middle
                            torch.tensor([[5, 0.1, 0.9], [0.9, 3, 0.1], [0.1, -0.3, 0.7], [0.7, 0.1, 0.1]])  # bad
        ]

        ground_truth = torch.tensor([2, 0, 1, 2])

        print('='*50)
        print("Class Weighted Distance 1.1")
        for model_output in model_outputs:
            loss = criterion(model_output=model_output, ground_truth=ground_truth)
            print(round(loss.item(), 3))
            
    def test_cdw_loss_v2(self):
        # =============================================================================
        # testing the loss function
        # this implementation has better results for the bad result
        # =============================================================================

        n_classes = 3
        criterion = ClassDistanceWeightedLoss(n_classes=n_classes)

        model_outputs = [ torch.tensor([[-0.1, -0.1, 5], [4, -0.1, -0.1], [-0.1, 6, -0.1], [-0.1, -0.1, 3]]), # good
                          torch.tensor([[0.1, 0.1, 0.9], [0.9, 0.1, 0.1], [0.1, 0.9, 0.1], [0.1, 0.1, 0.9]]), # middle
                          torch.tensor([[5, 0.1, 0.9], [0.9, 3, 0.1], [0.1, -0.3, 0.7], [0.7, 0.1, 0.1]])  # bad
        ]

        ground_truth = torch.tensor([2, 0, 1, 2])

        print('='*50)
        print("Class Weighted Distance 2")
        for model_output in model_outputs:
            loss = criterion(model_output=model_output, ground_truth=ground_truth)
            print(round(loss.item(), 3))
            
    def test_corn(self):
        # =============================================================================
        # testing the loss function
        # 3 classes, 2 outputs!!!
        # =============================================================================
        n_classes = 3
        criterion = CornLoss(n_classes=n_classes)

        # with 2 output neurons
        model_outputs = [ torch.tensor([[2, 1], [-0.3, 0.0], [0.8, 0.1], [2, 0.8]]), # good
                          torch.tensor([[0.1, 0.8], [0.1, 0.7], [0.8, 0.2], [0.7, 0.8]]), # mid
                          torch.tensor([[0.2, 0.1], [2, 0.7], [0.7, 0.8], [0.0, 0.3]]) # bad
        ]
        
        ground_truth = torch.tensor([2, 0, 1, 2])
        
        print('='*50)
        print("CORN")
        print("GrTr label", ground_truth)
        for model_output in model_outputs:
            m = criterion.get_predicted_label(model_output)
            print("pred label", m)
            loss = criterion(model_output=model_output, ground_truth=ground_truth)
            print("loss", round(loss.item(), 3))        

In [5]:
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.....
----------------------------------------------------------------------
Ran 5 tests in 0.031s

OK


Class Weighted Distance 1
0.027
0.383
1.756
Class Weighted Distance 1.1
0.027
0.383
1.756
Class Weighted Distance 2
0.082
1.148
5.268
CORN
GrTr label tensor([2, 0, 1, 2])
pred label tensor([2, 0, 1, 2])
loss 0.373
pred label tensor([1, 1, 1, 1])
loss 0.529
pred label tensor([1, 2, 1, 0])
loss 0.884
Kappa Loss
0.059
0.716
1.281
